## Identifiers

In [ ]:
import pickle
import sys
import spacy
from spacy import displacy
import benepar

In [ ]:
# Set up and store NLP
nlp = spacy.load("en_core_web_md")
benepar.download('benepar_en3')
nlp.add_pipe('benepar', config={'model': 'benepar_en3'})

with open('./nlp/nlp.pickle', 'wb') as f:
    pickle.dump(nlp, f)

In [ ]:
# Load NLP
#sys.path.append('../')

with open('./nlp/nlp.pickle', 'rb') as f:
    nlp = pickle.load(f)

In [ ]:
from app.classes.spec.primitive import *

from app.src.sentence_summarizer import SentenceSummarizer

summarizer = SentenceSummarizer(nlp)

### Testing it out

In [ ]:
from app.src.primitive_identifiers.dict_set_builder import DictSetBuilder


all_dicts = DictSetBuilder.build_all()

In [ ]:
from app.src.primitive_extractor import PrimitiveExtractor
from app.src.primitive_identifiers.obligation_event_identifier import ObligationEventIdentifier
from app.src.primitive_identifiers.power_event_identifier import PowerEventIdentifier
from app.src.primitive_identifiers.contract_event_identifier import ContractEventIdentifier
from app.src.primitive_identifiers.obligation_state_identifier import ObligationStateIdentifier
from app.src.primitive_identifiers.power_state_identifier import PowerStateIdentifier
from app.src.primitive_identifiers.contract_state_identifier import ContractStateIdentifier
from app.src.primitive_identifiers.event_scorer import EventScorer
from app.src.primitive_identifiers.primitive_identifier import IIdentifyPrimitives
from app.src.primitive_identifiers.state_name_scorer import StateNameScorer
from app.src.primitive_identifiers.variable_scorer import VariableScorer
from app.src.primitive_identifiers.similarity_calculator import SentenceSimilarityCalculator
from app.src.primitive_identifiers.sentence_preprocessor import SentencePreprocessor
from app.templates.meat_sale.templates import meat_sale_templates

preprocessor = SentencePreprocessor(nlp)
similarity_calculator = SentenceSimilarityCalculator(preprocessor)

# ObligationEvent
ob_event_scorer = EventScorer(all_dicts['obligation_events'])
obligation_sent_dict = meat_sale_templates['obligations']
ob_var_scorer = VariableScorer(nlp, obligation_sent_dict, similarity_calculator)
oe_identifier = ObligationEventIdentifier(ob_event_scorer, ob_var_scorer)

# PowerEvent
power_event_scorer = EventScorer(all_dicts['power_events'])
power_sent_dict = meat_sale_templates['powers']
power_var_scorer = VariableScorer(nlp, power_sent_dict, similarity_calculator)
pe_identifier = PowerEventIdentifier(power_event_scorer, power_var_scorer)

# ContractEvent
c_event_scorer = EventScorer(all_dicts['contract_events'])
ce_identifier = ContractEventIdentifier(c_event_scorer)

# ObligationState
ob_state_scorer = StateNameScorer(all_dicts['obligation_states'])
ob_var_scorer = VariableScorer(nlp, obligation_sent_dict, similarity_calculator)
os_identifier = ObligationStateIdentifier(ob_state_scorer, ob_var_scorer)

# PowerState
power_state_scorer = StateNameScorer(all_dicts['power_states'])
power_var_scorer = VariableScorer(nlp, power_sent_dict, similarity_calculator)
ps_identifier = PowerStateIdentifier(power_state_scorer, power_var_scorer)

# ContractState
c_state_scorer = StateNameScorer(all_dicts['contract_states'])
cs_identifier = ContractStateIdentifier(c_event_scorer)

ident_list: list[IIdentifyPrimitives] = [
    oe_identifier,
    pe_identifier,
    ce_identifier,
    os_identifier,
    ps_identifier,
    cs_identifier
]

primitive_extractor = PrimitiveExtractor(ident_list, 0.78)


In [ ]:

test_sentence = 'seller has resumed delivery of goods'
doc = nlp(test_sentence)

results = primitive_extractor.extract(doc)

for x in ident_list:
    next_res = x.identify(doc)
    print(type(next_res.primitive))
    print(next_res.score)


## Graph viz

In [13]:
from app.src.graph.graph_visualizer import GraphVisualizer
from app.src.graph.graph_builder import GraphBuilder
from app.classes.spec.symboleo_spec import PAtom

graph_builder = GraphBuilder()
digraph = graph_builder.build(PAtom)

gv = GraphVisualizer()


g = gv.create_viz(digraph.nodes)
g.show('nx.html')